In [6]:
!pip install pyttsx3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install pytelegrambotapi


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213956 sha256=8a44c053f345e43aae81c5110f7ec6aad6fa9e21841f17cd5a4f9838527d7d83
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pytelegrambotapi


In [14]:
import telebot
import string
import pickle
import re
import csv
import pandas as pd
import numpy as np


bot = telebot.TeleBot("6143785395:AAGNRjBthhXk9MZrwjSc1R2LjAKmpLXgH6s")

training = pd.read_csv('Training.csv')

reduced_data = training.groupby(training['prognosis']).max()

le = None
le2 = None
model, model2 = None, None

#load le from saved pickle after training
with open('label_encoder.pkl', 'rb') as f:
    le = pickle.load(f)

with open('label_encoder2.pkl', 'rb') as f:
    le2 = pickle.load(f)

with open('model1.pkl', 'rb') as f:
    model = pickle.load(f)

with open('model2.pkl', 'rb') as f:
    model2 = pickle.load(f)

symptomsDict = {symptom: index for index, symptom in enumerate(model.feature_names_in_)}
description_list = dict()

#initialize descriptions list
with open('symptom_Description.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        _description={row[0]:row[1]}
        description_list.update(_description)


states = dict() #store current user text
store = dict() #atore user answer
# symptoms = dict() #store symptoms list for user


FINAL = 6

def check_pattern(dis_list, _input):
    _input = _input.replace(' ', '_')

    pattern = f"{_input}"
    regexp = re.compile(pattern)

    pred_list = [item for item in dis_list if regexp.search(item)]
    return pred_list

def predict_from_symptoms(symptoms, use_other_model=False):
    inputVector = np.zeros(len(model.feature_names_in_)) #empty vector to be infilled
    for s in symptoms:
        inputVector[[symptomsDict[s]]] = 1 #fill symptoms vector
    # print(syptom_df)

    if not use_other_model:
      symptom_pred = model.predict([inputVector])
      disease = le.inverse_transform(symptom_pred)
      return disease

    else:
      symptom_pred = model2.predict([inputVector])
      disease = le2.inverse_transform(symptom_pred)
      return disease



def preprocess(symptom, duration):
    # Convert symptom to lowercase and remove punctuation
    symptom = symptom.lower().strip().translate(str.maketrans('', '', string.punctuation))

    # Convert duration to days
    duration_days = duration

    # Create a one-hot encoding of the symptom
    symptoms = ["fever", "cough", "fatigue", "headache", "loss of smell or taste"]
    symptom_one_hot = [int(symptom == s) for s in symptoms]

    # Combine the symptom and duration information into a single list
    data = [symptom_one_hot + [duration_days]]

    return data





@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.register_next_step_handler(message, start_repl)
    bot.reply_to(message, "Hello, I'm Carl, your health assistant. What's your name?")


@bot.message_handler(func=lambda message: True)
def start_repl(message):
    _id = message.chat.id
    store[_id] = {"name": message.text}
    bot.register_next_step_handler(message, get_symptoms)
    bot.reply_to(message, f"Hello, {store[_id]['name']}. What symptoms are you experiencing?")


@bot.message_handler(func=lambda message: True)
def get_symptoms(message):
    _id = message.chat.id
    latest_response = message.text

    if not "symptoms" in store[_id].keys():
        store[_id]['symptoms'] = [] #initialize empty


    if latest_response == "done":
        #done with listing symptoms

        bot.register_next_step_handler(message, get_symptom_days)
        bot.reply_to(message, "How many days has this occured?")
        return

    if latest_response in symptomsDict.keys():
        store[_id]['symptoms'].append(latest_response)
        bot.register_next_step_handler(message, get_symptoms )
        bot.reply_to(message, "Thank you for sharing your symptom. If you have any more to share, please go ahead, else type 'done'.")
        return

    else:
        #try to find symptoms by matching
        pred_list = check_pattern(symptomsDict.keys(), latest_response)

        if pred_list:
            resp  = "Did you mean any of the following? Reply with the updated name"
            for idx, i in enumerate(pred_list):
                resp += f"\n {idx}) {i}" #add each matched symptom
            bot.register_next_step_handler(message, get_symptoms)
            bot.reply_to_(message, resp)
            return


        else:
            #symptom does not match anything
            bot.reply_to(message, "Sorry this symptom is not in our database. Try again.")
            bot.register_next_step_handler(message, get_symptoms)
            return

@bot.message_handler(func=lambda message: True)
def get_symptom_days(message):
    _id = message.chat.id

    if message.text.isnumeric():
        print(store[_id]['symptoms'])
        store[_id]["duration"] = int(message.text)

        bot.register_next_step_handler(message, get_other_symptoms)
        bot.reply_to(message, f"I'm going to ask you some follow up questions just respond with yes/no. Type 'okay' to proceed")

    else:
        bot.register_next_step_handler(message, get_symptom_days)
        bot.reply_to(message, f"Response was not a number. Try again.")



@bot.message_handler(func=lambda message: True)
def get_other_symptoms(message):
    _id = message.chat.id



    if not "symptoms_given" in store[_id].keys():

        symptoms = store[_id]["symptoms"]
        present_disease  = predict_from_symptoms(symptoms)

        red_cols = reduced_data.columns
        symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]

        store[_id]['count'] = 0
        store[_id]["symptoms_given"] = symptoms_given


    if store[_id]['count'] >= len(store[_id]["symptoms_given"]):

        if message.text.lower() in ['yes', 'y']:
          store[_id]["symptoms"].append(store[_id]['symptoms_given'][-1]) #add last answer


        bot.reply_to(message, "Done collecting symptoms. Type 'okay' to proceed.")
        bot.register_next_step_handler(message, get_prediction)

    else:
        cnt = store[_id]['count']
        if cnt > 0:
            if message.text.lower() in ['yes', 'y']:
              store[_id]["symptoms"].append(store[_id]['symptoms_given'][cnt]) #add symptom


        next_symptom = store[_id]["symptoms_given"][cnt]
        bot.reply_to(message, f"{next_symptom} ? ")

        bot.register_next_step_handler(message, get_other_symptoms)
        store[_id]['count'] += 1
        return


@bot.message_handler(func=lambda message: True)
def get_prediction(message):
    _id = message.chat.id

    symptoms = store[_id]["symptoms"]

    pred1  = predict_from_symptoms(symptoms)[0]
    # pred2  = predict_from_symptoms(symptoms, True)[0]
    # print(pred1, pred2, description_list)
    # print(pred1, type(pred1), description_list, type(description_list))
    # resp = f"You may have {pred1} or {pred2}. \n {description_list[pred1]} \n {description_list[pred2]}. "
    resp = f"You may have {pred1}. \n {description_list[pred1]}. "
    bot.register_next_step_handler(message, send_welcome)
    bot.reply_to(message, resp)


    #clean up?
    del store[_id]

    import smtplib
    import ssl
    from email.message import EmailMessage

    email_sender = 'joshua.healthchatbot@gmail.com'
    email_password = 'vgjphuxoxavaadri'
    email_receiver = 'jjsam106@gmail.com'

    subject = 'Check out my new video!'
    body= pred1

    em = EmailMessage()
    em['From'] = email_sender
    em['To'] = email_receiver
    em['Subject'] = subject
    em.set_content(body)

    context = ssl.create_default_context()

    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(email_sender, email_password)
        smtp.sendmail(email_sender, email_receiver, em.as_string())





bot.infinity_polling()




2023-06-19 00:11:58,307 (__init__.py:966 MainThread) ERROR - TeleBot: "Infinity polling: polling exited"
ERROR:TeleBot:Infinity polling: polling exited
2023-06-19 00:11:58,310 (__init__.py:968 MainThread) ERROR - TeleBot: "Break infinity polling"
ERROR:TeleBot:Break infinity polling
